In [3]:
# 1 import and save

import csv
import os
import pickle


csv_file_path = "./Diabetes-PDFs-Outputs/all_papers.csv"
output_full_dict_path = "./data_process/ori_data/full_text_dict.pkl"
output_abstract_dict_path = "./data_process/ori_data/abstract_dict.pkl"
output_section1_dict_path = "./data_process/ori_data/introduction_dict.pkl"
output_section2_dict_path = "./data_process/ori_data/method_dict.pkl"
output_section3_dict_path = "./data_process/ori_data/result_dict.pkl"

# section1，2，3 : introduction，method 和 results 


full_text_dict = {}                 
abstract_dict = {}
section1_dict = {}
section2_dict = {}
section3_dict = {}

# read csv
with open(csv_file_path, 'r', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    for idx, row in enumerate(reader):
        paper_id = f"paper_{idx + 1}"
        

        path_1 = os.path.join('./Diabetes-PDFs-Outputs/', row['PDF name'], 'pickle', row['abstract'])
        with open(path_1, 'rb') as file:
            abstract = pickle.load(file)
        

        path_2 = os.path.join('./Diabetes-PDFs-Outputs/', row['PDF name'], 'pickle', row['section 1'])
        with open(path_2, 'rb') as file:
            section1 = pickle.load(file)

        path_3 = os.path.join('./Diabetes-PDFs-Outputs/', row['PDF name'], 'pickle', row['section 2'])
        with open(path_3, 'rb') as file:
            section2 = pickle.load(file)
        
        path_4 = os.path.join('./Diabetes-PDFs-Outputs/', row['PDF name'], 'pickle', row['section 3'])
        with open(path_4, 'rb') as file:
            section3 = pickle.load(file)
        

        full_text = {
            'abstract': abstract,
            'intro': section1,
            'method': section2,
            'result': section3,
            'paper_name': row['article']
        }
        

        full_text_dict[paper_id] = full_text
        abstract_dict[paper_id] = abstract
        section1_dict[paper_id] = section1
        section2_dict[paper_id] = section2
        section3_dict[paper_id] = section3


with open(output_full_dict_path, 'wb') as file:
    pickle.dump(full_text_dict, file)
with open(output_abstract_dict_path, 'wb') as file:
    pickle.dump(abstract_dict, file)
with open(output_section1_dict_path, 'wb') as file:
    pickle.dump(section1_dict, file)
with open(output_section2_dict_path, 'wb') as file:
    pickle.dump(section2_dict, file)
with open(output_section3_dict_path, 'wb') as file:
    pickle.dump(section3_dict, file)

print("successful")


文件已成功保存。


In [5]:
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.8/328.8 kB 4.2 MB/s eta 0:00:0000:0100:01


In [7]:
# use api first, change later

from openai import OpenAI

client = OpenAI(
  base_url = "https://integrate.api.nvidia.com/v1",
  api_key = "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"
)

completion = client.chat.completions.create(
  model="meta/llama3-70b-instruct",
  messages=[{"role":"user","content":"generate a literature review about risk factors of Type 2 diabetes, the paper contains more than 5000 words"}],
  temperature=0.5,
  top_p=1,
  max_tokens=8000, # 8192 max
  stream=True
)

abs_str = ''
for chunk in completion:
  if chunk.choices[0].delta.content is not None:
    abs_str += chunk.choices[0].delta.content.replace('\n', ' ')
  
# print(abs_str)


Here is a literature review on the risk factors of Type 2 diabetes, exceeding 5000 words:  **Title:** Risk Factors of Type 2 Diabetes: A Comprehensive Review  **Introduction:** Type 2 diabetes (T2D) is a complex and multifactorial disease that has become a major public health concern worldwide. It is estimated that over 463 million people globally suffer from diabetes, with T2D accounting for approximately 90% of all diabetes cases (International Diabetes Federation, 2019). The prevalence of T2D is increasing rapidly, particularly in low- and middle-income countries, where it is expected to reach 578 million by 2030 (International Diabetes Federation, 2019). The prevention and management of T2D require a thorough understanding of its risk factors. This literature review aims to provide a comprehensive overview of the risk factors associated with T2D, including modifiable and non-modifiable factors.  **Non-Modifiable Risk Factors:**  ### 1. Age  Advancing age is a significant risk facto

In [8]:
# Generate a summary for each specific article
# Use llama3 to generate a summary of no more than 100 words for the summary and introduction for topic classification

import pickle

pkl_file_path = './data_process/ori_data/full_text_dict.pkl'

with open(pkl_file_path, 'rb') as file:
    paper_dict = pickle.load(file)


paper_info = {}
for key, value in paper_dict.items():
    paper_info[key] = {
        "abs_intro": value['intro'] + value['method'],
        "llm_abs": '' 
    }

In [10]:
import tiktoken

# calculate the length of token 
def count_tokens(text, model="gpt-3.5-turbo"):
    encoding = tiktoken.encoding_for_model(model)
    tokens = encoding.encode(text)
    return len(tokens)

# split text 
def split_text(text, max_tokens=5000, model="gpt-3.5-turbo"):
    encoding = tiktoken.encoding_for_model(model)
    tokens = encoding.encode(text)
    
    if len(tokens) <= max_tokens:
        return [text]
    
    splits = []
    start = 0
    while start < len(tokens):
        end = start + max_tokens
        split_tokens = tokens[start:end]
        split_text = encoding.decode(split_tokens)
        splits.append(split_text)
        start = end

    return splits

# process text
def summarize_text(text, client, model="meta/llama3-70b-instruct", max_tokens=8192):
    splits = split_text(text, max_tokens=4000)
    summaries = []
    
    for split in splits:
        completion = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": f"Summarize the following text: {split}"}],
            temperature=0.5,
            top_p=1,
            max_tokens=4000,  # Allow some buffer for response tokens
            stream=True
        )
        summary = ""
        for chunk in completion:
            if chunk.choices[0].delta.content is not None:
                summary += chunk.choices[0].delta.content.replace('\n', ' ')
        summaries.append(summary)
    
    return summaries

def combine_summaries(summaries):
    combined_summary = " ".join(summaries)
    return combined_summary

abs_dict = {}

for key, value in paper_info.items():
    text = value['abs_intro']
    while True:
        # generate summary first
        summaries = summarize_text(text, client)

        # generate summary with multiple times until the length of tokens is smaller than 8000 words
        encoding = tiktoken.encoding_for_model('gpt-3.5-turbo')
        tokens = encoding.encode(text)
        numtokens = len(tokens)
        if numtokens < 8000:
            break
        else:
            text = ''.join(summaries)

    final_summary = combine_summaries(summaries)

    # generate text within 100 words
    print(key)
    print(final_summary)
    print('-' * 100)
    abs_dict[key] = final_summary

paper_1
The text discusses the relationship between diet quality and the risk of developing type 2 diabetes mellitus (T2DM). The authors highlight the growing incidence of T2DM and its association with unhealthy diets, particularly those high in processed foods and added sugars. In contrast, adherence to certain dietary patterns, such as the Mediterranean diet, has been shown to have a protective effect against T2DM.  The authors introduce the concept of nutrient density, which refers to the ratio of nutrients to energy in a diet. They discuss various diet quality indices (DQIs), including the Alternative Healthy Eating Index 2010 (AHEI-2010) and the Nutrient-Rich Food Index 9.3 (NRF9.3), which are used to assess the quality of a diet.  The study aims to evaluate the association between the NRF9.3 index and the risk of developing T2DM in patients with coronary heart disease (CHD). The authors used data from the CORDIOPREV study, a randomized controlled trial that included 1002 patients

In [9]:
# For each specific summary, 
# use llama3 to generate a summary of no more than 100 words for the summary and introduction for topic classification

one_abs = {}
for key, value in abs_dict.items():
    # Construct summaries of less than 100 words for topic classification
    completion = client.chat.completions.create(
            model='meta/llama3-70b-instruct',
            messages=[{"role": "user", "content": f"You are a scholar in the medical field specializing in the study of type 2 diabetes. You have excellent knowledge of the causes, risks, and symptoms of diabetes. Now, you need to synthesize existing diabetes research papers into a clear and well-organized review. I will provide you with relevant sections of these papers, and you need to generate summaries to facilitate content and theme classification. The requirements are as follows: \
1. Extract content to generate a summary that clearly conveys the core content. \
2. Ensure that all content in the summary can be found in the provided sections and is accurate. \
3. The generated summary must be clear, concise, logical, and coherent, using an academic tone. \
4. The word limit for the generated summary is 100 words. \
5. Just output the final summary without other unnecessary words. \
The text is {value}"}],
            temperature=0.5,
            top_p=1,
            max_tokens=4000,  # Allow some buffer for response tokens
            stream=True
        )
    abs_summary = ""
    for chunk in completion:
        if chunk.choices[0].delta.content is not None:
            abs_summary += chunk.choices[0].delta.content.replace('\n', ' ')

    one_abs[key] = abs_summary
    print(abs_summary)



This review examines the relationship between diet quality and the risk of type 2 diabetes mellitus (T2DM). The authors highlight the benefits of adhering to dietary patterns such as the Mediterranean diet, which reduces T2DM risk. They introduce the concept of nutrient density and diet quality indices (DQIs), including the Nutrient-Rich Food Index 9.3 (NRF9.3), which predicts chronic disease risk. A study found that a higher NRF9.3 score is associated with a lower T2DM risk, suggesting that promoting nutrient-dense diets may be an effective T2DM prevention strategy.
This study investigated risk factors for inflammation in type 2 diabetes mellitus (T2DM) patients with acute ischemic stroke (AIS). Elevated C-reactive protein (CRP) and lipoprotein-associated phospholipase A2 (Lp-PLA2) levels, indicators of inflammation and atherosclerosis, were associated with increased AIS risk in T2DM patients. The study suggests that controlling diabetes and inflammation may reduce stroke risk and sev

In [11]:
# classify topic

def construct_prompt(text_dict):

    prompt = """ \
    You are a medical expert specializing in analyzing and summarizing medical research texts. Your task is to review a collection of medical texts and provide a concise summary of the main topics covered in each text. Follow these guidelines: \
1. **Analyze each text individually**: Examine each text and identify the main medical topics discussed. \
2. **Summarize the topics**: For each text, generate a brief summary (no more than 30 words) that captures the core medical topics and themes. \
3. **Stop outputting other word, just output the json format. \
4. **Output format**: Provide the results in JSON format, with each text's summary represented as an object with the structure: \
{ \
  "ID": "summary of topics"  \
}   """

    for idx, (key, value) in enumerate(text_dict.items(), start=1):
        prompt += f"Text {idx} (ID: {key}):\n{value}\n\n"

    prompt += "Please summarize the topics covered in each text above."
    
    return prompt

# example
text_dict = one_abs

prompt = construct_prompt(text_dict)
print(prompt)

# generate topic

completion = client.chat.completions.create(
            model='meta/llama3-70b-instruct',
            messages=[{"role": "user", "content": prompt}],
            temperature=0.5,
            top_p=1,
            max_tokens=4000,  # Allow some buffer for response tokens
            stream=True
        )
summary = ""
for chunk in completion:
    if chunk.choices[0].delta.content is not None:
        summary += chunk.choices[0].delta.content.replace('\n', ' ')

print('--')
print(summary)



     You are a medical expert specializing in analyzing and summarizing medical research texts. Your task is to review a collection of medical texts and provide a concise summary of the main topics covered in each text. Follow these guidelines: 1. **Analyze each text individually**: Examine each text and identify the main medical topics discussed. 2. **Summarize the topics**: For each text, generate a brief summary (no more than 30 words) that captures the core medical topics and themes. 3. **Stop outputting other word, just output the json format. 4. **Output format**: Provide the results in JSON format, with each text's summary represented as an object with the structure: {   "ID": "summary of topics"  }   Text 1 (ID: paper_1):
This review examines the relationship between diet quality and the risk of type 2 diabetes mellitus (T2DM). The authors highlight the benefits of adhering to dietary patterns such as the Mediterranean diet, which reduces T2DM risk. They introduce the concept o

In [11]:
# Only based on the abstract
topic_dic = {}

for key, value in abs_dict.items():
    text = value
    prompt = """
You are now a professional diabetes research doctor, working on organizing diabetes risk research. I will provide you with the abstracts of various articles. Please determine which direction of diabetes research each abstract corresponds to. The specific requirements are as follows:
There are three categories: single risk factor study, meta-analysis of a single risk factor, and meta-analysis of multiple risk factors.
A meta-analysis of a single risk factor focuses on a specific risk factor, evaluating its impact on diabetes risk by synthesizing data from multiple studies. A single risk factor study focuses on a single risk factor within one study, typically an original study. A meta-analysis of multiple risk factors considers multiple risk factors simultaneously, evaluating their impact on diabetes risk comprehensively and analyzing their interactions.
The category of the abstract must be one of these three categories, or very close to one of them. In such cases, the abstract should be classified under that category.
The final result should only output a number representing the category: 1 for single risk factor study, 2 for meta-analysis of a single risk factor, and 3 for meta-analysis of multiple risk factors. For example, if the abstract belongs to a single risk factor study, output: 1
\n\n Abstract:\n\n{}
""".format(text)
    completion = client.chat.completions.create(
            model='meta/llama3-70b-instruct',
            messages=[{"role": "user", "content": prompt}],
            temperature=0.5,
            top_p=1,
            max_tokens=4000,  # Allow some buffer for response tokens
            stream=True
        )
    summary = ""
    for chunk in completion:
        if chunk.choices[0].delta.content is not None:
            summary += chunk.choices[0].delta.content.replace('\n', ' ')
    
    topic_dic[key] = summary
    print(summary)

    


1
1
2
1
1
2


In [13]:
print(topic_dic)

{'paper_1': '1', 'paper_2': '1', 'paper_3': '2', 'paper_4': '1', 'paper_5': '2', 'paper_6': '2'}


In [12]:
# Generate a review first
import tiktoken
def count_tokens(text, model="gpt-3.5-turbo"):
    encoding = tiktoken.encoding_for_model(model)
    tokens = encoding.encode(text)
    return len(tokens)

# Process text in segments
def split_text(text, max_tokens=5000, model="gpt-3.5-turbo"):
    encoding = tiktoken.encoding_for_model(model)
    tokens = encoding.encode(text)
    
    if len(tokens) <= max_tokens:
        return [text]
    
    splits = []
    start = 0
    while start < len(tokens):
        end = start + max_tokens
        split_tokens = tokens[start:end]
        split_text = encoding.decode(split_tokens)
        splits.append(split_text)
        start = end
    
    return splits

def summarize_section(text, section_name, client, model="meta/llama3-70b-instruct", max_tokens=4000):
    splits = split_text(text, max_tokens=max_tokens)
    summaries = []
    
    for split in splits:
        prompt = f"Given the following text from a research paper, generate a concise and clear summary for the {section_name} section. \
            Ensure that the summary accurately reflects the key points and findings from the provided text.\n\n \
            Requirements: \
            1. Only output the summary information, such as [abstract: just output the generated summary] \
            2. Every informatin must be from the original text \
            The text provided is below: \
            Text:\n{split}\n\nSummary:" 
        completion = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            temperature=0.5,
            top_p=1,
            max_tokens=max_tokens - 200,
            stream=True
        )
        summary = ""
        for chunk in completion:
            if chunk.choices[0].delta.content is not None:
                summary += chunk.choices[0].delta.content.replace('\n', ' ')
        summaries.append(summary)
    
    final_summary = " ".join(summaries)
    return final_summary

def generate_review(papers, client, model="meta/llama3-70b-instruct"):
    review = {}
    
    for paper_id, sections in papers.items():
        review[paper_id] = {}
        for section_name in ['abstract', 'intro', 'method', 'methods', 'result']:
            if section_name in sections:
                text = sections[section_name]
                summary = summarize_section(text, section_name, client, model)
                review[paper_id][section_name] = summary
    
    return review



In [13]:
# generate literature reviews
import pickle


pkl_file_path = './data_process/ori_data/full_text_dict.pkl'

with open(pkl_file_path, 'rb') as file:
    paper_dict = pickle.load(file)

papers = paper_dict
review = generate_review(papers, client)

# save
resultdic = {}
for paper_id, sections in review.items():
    print(f"Review for {paper_id}:")
    resultdic[paper_id] = {}
    for section_name, summary in sections.items():
        resultdic[paper_id][section_name] = summary
        print(f"{section_name.capitalize()}: {summary}\n")


Review for paper_1:
Abstract: Abstract: This study investigated the relationship between changes in diet quality and the risk of developing type 2 diabetes mellitus (T2DM) in patients with coronary heart disease (CHD). In a 5-year dietary intervention, patients who showed greater improvement in nutrient-dense diet quality had over 50% lower risk of developing T2DM compared to those with the lowest improvement. These findings suggest that promoting high-quality dietary patterns may help reduce the incidence of T2DM in patients with CHD.

Intro: Abstract: Type 2 diabetes mellitus (T2DM) is a growing chronic disease closely linked to overweight, obesity, and unhealthy diets. Adherence to high-quality dietary patterns, such as the Mediterranean diet, DASH, and vegetarian diets, has been shown to prevent T2DM incidence. The nutrient density of a diet, measured by diet quality indices (DQIs), is a good indicator of diet quality. The Alternative Healthy Eating Index 2010 (AHEI-2010) and the N

In [14]:
resultdic = {}
for paper_id, sections in review.items():
    print(f"Review for {paper_id}:")
    resultdic[paper_id] = {}
    for section_name, summary in sections.items():
        resultdic[paper_id][section_name] = summary
        print(f"{section_name.capitalize()}: {summary}\n")

Review for paper_1:
Abstract: Abstract: This study investigated the relationship between changes in diet quality and the risk of developing type 2 diabetes mellitus (T2DM) in patients with coronary heart disease (CHD). In a 5-year dietary intervention, patients who showed greater improvement in nutrient-dense diet quality had over 50% lower risk of developing T2DM compared to those with the lowest improvement. These findings suggest that promoting high-quality dietary patterns may help reduce the incidence of T2DM in patients with CHD.

Intro: Abstract: Type 2 diabetes mellitus (T2DM) is a growing chronic disease closely linked to overweight, obesity, and unhealthy diets. Adherence to high-quality dietary patterns, such as the Mediterranean diet, DASH, and vegetarian diets, has been shown to prevent T2DM incidence. The nutrient density of a diet, measured by diet quality indices (DQIs), is a good indicator of diet quality. The Alternative Healthy Eating Index 2010 (AHEI-2010) and the N

In [15]:
 # 拼接每个论文的相同章节内容
merged_sections = {}
for paper_id, sections in review.items():
    # 遍历每篇论文的章节
    for section_name, text in sections.items():
        # 如果章节名在合并字典中，则将文本拼接到对应的字段中
        if section_name in merged_sections:
            merged_sections[section_name] += text + "\n\n"
        else: 
            merged_sections[section_name] = text


In [16]:
import re
import tiktoken

def filter_first_occurrence(text):
    pattern = re.compile(r'Here .*? abstracts')
    text = pattern.sub('', text, count=1)

    # 'Here --- sections' format
    pattern = re.compile(r'Here .*? sections')

    filtered_text = pattern.sub('', text, count=1)

    pattern = re.compile(r'Here .*? section')

    filtered_text = pattern.sub('', filtered_text, count=1)

    return filtered_text

def count_tokens(text, model="gpt-3.5-turbo"):
    encoding = tiktoken.encoding_for_model(model)
    tokens = encoding.encode(text)
    return len(tokens)

# 分段处理文本
def split_text(text, max_tokens=5000, model="gpt-3.5-turbo"):
    encoding = tiktoken.encoding_for_model(model)
    tokens = encoding.encode(text)
    
    if len(tokens) <= max_tokens:
        return [text]
    
    splits = []
    start = 0
    while start < len(tokens):
        end = start + max_tokens
        split_tokens = tokens[start:end]
        split_text = encoding.decode(split_tokens)
        splits.append(split_text)
        start = end
    
    return splits

def sum_section(text, section_name, client, model="meta/llama3-70b-instruct", max_tokens=4000):
    splits = split_text(text, max_tokens=max_tokens)
    summaries = []
    
    for split in splits:
        prompt = f"""
        You are a scholar in the medical field specializing in the study of type 2 diabetes. You have excellent knowledge of the causes, risks, and symptoms of diabetes. Now, you need to synthesize existing diabetes research papers into a clear and well-organized review. I will provide you with relevant sections of these papers, and you need to generate summaries to facilitate content and theme classification. The requirements are as follows:
        Given the following text from a research paper, generate a concise and clear summary for the {section_name} section. \
            Ensure that the summary accurately reflects the key points and findings from the provided text.\n\n \
            Requirements: \
            1. Extract content to generate a summary that clearly conveys the core content.
            2. Ensure that all content in the summary can be found in the provided sections and is accurate.
            3. The generated summary must be clear, concise, logical, and coherent, using an academic tone.
            4. Every informatin must be from the original text \
            5. The length of the generated word must be between 500 and 1000 words. \
            6. Only output the summary results, and do not enter any irrelevant words or expressions, such as 'Here is a concise and clear summary of the xxx section' is limited \
            7. The content should be summarized and stated in the tone of an academic review without other unnessary description. \
            The text provided is below: \
            Text:\n{split}"""
        
        completion = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            temperature=0.8,
            top_p=1,
            max_tokens=max_tokens - 200,
            stream=True
        )
        summary = ""
        for chunk in completion:
            if chunk.choices[0].delta.content is not None:
                summary += chunk.choices[0].delta.content.replace('\n', ' ')
        # print(summary)
        # print('-' * 100)
        summaries.append(summary)
    
    final_summary = " ".join(summaries)
    return final_summary

def generate_survey(papers, client, model="meta/llama3-70b-instruct"):
    review = {}
    
    for section_name, text in papers.items():
        summary = sum_section(text, section_name, client, model)
        review[section_name] = summary
    
    return review

# 生成综述

review = generate_survey(merged_sections, client)

# save results
resultdic = {}
for section_name, summary in review.items():
    resultdic[section_name] = summary
    print(f"{section_name.capitalize()}: {filter_first_occurrence(summary)}\n")

Abstract: This review synthesizes evidence from multiple studies to provide insights into the risk factors, prevention, and management of type 2 diabetes mellitus (T2DM). The findings highlight the significance of dietary patterns, nutrient intake, and lifestyle factors in the development and prevention of T2DM.  In patients with coronary heart disease, promoting high-quality dietary patterns may reduce the incidence of T2DM by over 50%. Additionally, habitual sugar-sweetened beverage consumption is associated with an increased risk of metabolic syndrome and its component conditions, including obesity, T2DM, and hypertension. Moderate dietary intakes of antioxidants such as vitamin C, vitamin E, and β-carotene may also play a role in T2DM prevention by reducing insulin resistance.  Furthermore, the review identifies high-sensitivity C-reactive protein and lipoprotein-associated phospholipase A2 as potential biomarkers for acute ischemic stroke risk in T2DM patients. The evidence sugges